<a href="https://colab.research.google.com/github/msc-acse/acse-9-independent-research-project-Garethlomax/blob/baseline_hybrid/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !ls

%cd /content/drive/My \Drive/masters_project/python_modules/MovingMNIST-master

# all torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from torchvision import models

import matplotlib.pyplot as plt
import h5py

# importing moving mnist test set.
from MovingMNIST import MovingMNIST

%cd /content/drive/My \Drive/masters_project/python_modules/pytorch-summary-master
from torchsummary import summary

# %cd /content/drive/My \Drive/masters_project/python_modules/pytorch_modelsize-master

%cd /content/drive/My \Drive/masters_project/python_modules/pytorchvis-master

!pip install torchviz

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True




/content/drive/My Drive/masters_project/python_modules/MovingMNIST-master
/content/drive/My Drive/masters_project/python_modules/pytorch-summary-master
[Errno 2] No such file or directory: '/content/drive/My Drive/masters_project/python_modules/pytorchvis-master'
/content/drive/My Drive/masters_project/python_modules/pytorch-summary-master
     |████████████████████████████████| 51kB 3.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/2a/c2/c5/b8b4d0f7992c735f6db5bfa3c5f354cf36502037ca2b585667
Successfully built torchviz


# MODEL 

In [0]:
# alex_net = models.A
alex = models.alexnet(pretrained = True)

# # make conditioned - give the decoder the last output of encoder as input.
# class Enc_Dec(nn.Module):
#     def __init__(self):
#         super(Enc_Dec, self).__init__()
        
#         self.

In [36]:
alex.features[0] = nn.Conv2d(1,64, kernel_size = (11,11), stride = (4,4), padding = (2,2))
for i, params in enumerate(alex.parameters()):
    if (i==1):
        print(params.data)



tensor([-0.0256, -0.0375,  0.0735,  0.0874,  0.0628,  0.0286,  0.0732,  0.0381,
         0.0640,  0.0514,  0.0553,  0.0162,  0.0231, -0.0360, -0.0778, -0.0168,
        -0.0294, -0.0403,  0.0394, -0.0572, -0.0486,  0.0241, -0.0201,  0.0023,
         0.0849, -0.0769, -0.0115,  0.0853, -0.0339, -0.0743, -0.0030, -0.0679,
        -0.0410, -0.0473,  0.0887,  0.0899,  0.0318,  0.0529,  0.0029, -0.0144,
        -0.0043, -0.0799,  0.0124,  0.0035, -0.0334, -0.0744, -0.0087, -0.0124,
        -0.0393,  0.0374, -0.0581, -0.0861, -0.0263,  0.0296,  0.0029, -0.0737,
        -0.0097,  0.0551,  0.0031,  0.0278, -0.0373, -0.0124, -0.0591,  0.0813])
